<a href="https://colab.research.google.com/github/EOKELLO/US-president-election-predictions/blob/master/Copy_of_%22Moringa_Data_Science_Prep_W2_Independent_Project_2019_06_Elizabeth_Okello_SQL_Notebook%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#WE CONNECT TO OUR DATABASE
#we first load the sql extension into our environment
%load_ext sql

#Then connect to our in memory sqlite database
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [0]:
#Import python libraries that we will need in this notebook
import csv
import pandas as pd


In [0]:
#loading the table from a local CSV file
#
with open('Elections.csv', 'r')as f:
      Elections = pd.read_csv(f, index_col=0, encoding='utf-8' )

In [0]:
#Then storing it in an SQL table in our sqlite memory
#
%sql DROP TABLE if EXISTS Elections;
%sql PERSIST Elections
%sql SELECT * FROM Elections LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [0]:
#loading the table from a local CSV file
#
with open('Population_state.csv', 'r')as f:
      Population_state = pd.read_csv(f, index_col=0, encoding='utf-8' )

In [0]:
#Then storing it in an SQL table in our sqlite memory
#
%sql DROP TABLE if EXISTS Population_state;
%sql PERSIST Population_state
%sql SELECT * FROM Population_state LIMIT 5;


 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [0]:
# convert state column items in elections dataset to uppercase
%%sql
UPDATE Elections set(State) = UPPER(State);

 * sqlite://
51 rows affected.


[]

In [0]:
#view to see the effect of the update on the table
%%sql
SELECT * FROM Elections
LIMIT 5;

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55


In [0]:
# We now join the two tables at the state column
#
%%sql
SELECT Elections.State ,Population , GrandElectors 
FROM Elections 
INNER JOIN Population_state ON Elections.State = Population_state.State
LIMIT 5;


 * sqlite://
Done.


State,Population,GrandElectors
ALABAMA,4858979,9
ALASKA,738432,3
ARIZONA,6828065,11
ARKANSAS,2978204,6
CALIFORNIA,39144818,55


In [0]:
%%sql
CREATE TABLE interim AS
SELECT GrandElectors, Population, UPPER(Population_state. State) AS STATEPOP, UPPER(Elections.State) AS STATE, Population_state.State AS STATEINPOP, Elections.State
FROM Elections INNER JOIN Population_state 
ON UPPER(Elections.State) = UPPER(Population_state.State);

SELECT * FROM interim
LIMIT 5;


 * sqlite://
Done.
Done.


GrandElectors,Population,STATEPOP,STATE,STATEINPOP,State:1
9,4858979,ALABAMA,ALABAMA,ALABAMA,ALABAMA
3,738432,ALASKA,ALASKA,ALASKA,ALASKA
11,6828065,ARIZONA,ARIZONA,ARIZONA,ARIZONA
6,2978204,ARKANSAS,ARKANSAS,ARKANSAS,ARKANSAS
55,39144818,CALIFORNIA,CALIFORNIA,CALIFORNIA,CALIFORNIA


In [0]:
%%sql 
DROP TABLE interim

 * sqlite://
Done.


[]

In [0]:

%sql DROP TABLE if EXISTS Elections;
%sql PERSIST Elections;


 * sqlite://
Done.
 * sqlite://


'Persisted elections'

In [0]:
#to change the name of the "District of Columbia" state to its short version "DC".
#Here we use the update records syntax as below
#
%%sql
UPDATE interim
SET STATEPOP = "DC", STATE = "DC", STATEINPOP = "DC"
WHERE Population = 672228 ;


SELECT *
FROM interim
where STATE = "DC";




 * sqlite://
1 rows affected.
Done.


GrandElectors,Population,STATEPOP,STATE,STATEINPOP,State:1
3,672228,DC,DC,DC,DISTRICT OF COLUMBIA


In [0]:
%%sql
SELECT *, (Population/GrandElectors) AS "Ratio"
FROM interim
ORDER BY Ratio DESC
LIMIT 5;


 * sqlite://
Done.


GrandElectors,Population,STATEPOP,STATE,STATEINPOP,State:1,Ratio
38,27469114,TEXAS,TEXAS,TEXAS,TEXAS,722871
55,39144818,CALIFORNIA,CALIFORNIA,CALIFORNIA,CALIFORNIA,711723
29,20271272,FLORIDA,FLORIDA,FLORIDA,FLORIDA,699009
29,19795791,NEW YORK,NEW YORK,NEW YORK,NEW YORK,682613
15,10042802,NORTH CAROLINA,NORTH CAROLINA,NORTH CAROLINA,NORTH CAROLINA,669520


In [0]:
%%sql
select A.STATE, A.GrandElectors, A.Population, sum(B.GrandElectors), sum(B.Population)
from interim AS A
INNER JOIN interim AS B ON ( A.Population >= B.Population)
GROUP BY  A.Population
ORDER BY A.GrandElectors asc
limit 41

 * sqlite://
Done.


STATE,GrandElectors,Population,sum(B.GrandElectors),sum(B.Population)
WYOMING,3,586107,3,586107
VERMONT,3,626042,6,1212149
DC,3,672228,9,1884377
ALASKA,3,738432,12,2622809
NORTH DAKOTA,3,756927,15,3379736
SOUTH DAKOTA,3,858469,18,4238205
DELAWARE,3,945934,21,5184139
MONTANA,3,1032949,24,6217088
RHODE ISLAND,4,1056298,28,7273386
MAINE,4,1329328,32,8602714


In [0]:
%%sql
select (sum(GrandElectors) / 2)
from interim

 * sqlite://
Done.


(sum(GrandElectors) / 2)
269


In [0]:
%%sql
select A.STATE, A.GrandElectors, A.Population, sum(B.GrandElectors), sum(B.Population)
from interim AS A
INNER JOIN interim AS B ON ( A.Population <= B.Population)
GROUP BY  A.Population
ORDER BY A.GrandElectors desc
limit 41

 * sqlite://
Done.


STATE,GrandElectors,Population,sum(B.GrandElectors),sum(B.Population)
CALIFORNIA,55,39144818,55,39144818
TEXAS,38,27469114,93,66613932
NEW YORK,29,19795791,151,106680995
FLORIDA,29,20271272,122,86885204
PENNSYLVANIA,20,12802503,191,132343493
ILLINOIS,20,12859995,171,119540990
OHIO,18,11613423,209,143956916
MICHIGAN,16,9922576,256,174137154
GEORGIA,16,10214860,225,154171776
NORTH CAROLINA,15,10042802,240,164214578
